In [44]:
import random 
random.seed(17)
import pandas as pd 
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
MAX_TOWERS = 6
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
np.random.seed(17)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

/anaconda2/envs/env/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [13]:
def get_train():
    train_main = pd.read_csv("../data/task2/train_2.7.csv", encoding="cp1251")
    train_aux_coords = pd.read_csv("../data/task2_additional/coords_train_2.1.csv", encoding="cp1251")
    train_aux_frac = pd.read_csv("../data/task2_additional/frac_train_2.csv", encoding="cp1251")
    train_aux_gdis = pd.read_csv("../data/task2_additional/gdis_train2.2.csv", encoding="cp1251")
    return train_main

train = get_train()

def get_test():
    test_main = pd.read_csv("../data/task2/test_2.9.csv", encoding="cp1251")
    test_aux_coords = pd.read_csv("../data/task2_additional/coords_test_2.1.csv", encoding="cp1251")
    test_aux_frac = pd.read_csv("../data/task2_additional/frac_test_2.csv", encoding="cp1251")
    test_aux_gdis = pd.read_csv("../data/task2_additional/gdis_test2.2.csv", encoding="cp1251")
    return test_main

test = get_test()

In [21]:
#sort by converted date and group
def convert_and_sort(df):
    df["Дата"] =  df["Дата"].apply(pd.to_datetime)
    df.sort_values(by=["Скважина", "Дата"], inplace=True)

convert_and_sort(train)
convert_and_sort(test)

In [23]:
#find columns with same value(useless) in train
def get_non_useful(df):
    non_useful_columns = []
    for c in df.columns:
        null_columns = df[df[c].isnull()]
        if len(null_columns)== len(df):
            non_useful_columns.append(c)
    return non_useful_columns

def drop_non_useful(train, test):
    non_useful = set(get_non_useful(train)) |set(get_non_useful(test))
    print("%s dropped"% non_useful)
    train.drop(list(non_useful), axis=1, inplace=True)
    test.drop(list(non_useful), axis=1, inplace=True)

drop_non_useful(train, test)

{'Станок-качалка', 'Агент закачки', 'Фирма ГРП', 'Тип газосепаратора'} dropped


In [24]:
def get_target(df, column="Нефть, т"):
    target = df[column]
    df.drop([column], axis=1, inplace=True)
    print("%s dropped"% column)
    return target

#remove target from train
y_train = get_target(train)

Нефть, т dropped


In [25]:
#drop non present columns in test
def drop_not_present(train, test):
    absent_columns = list(set(train.columns) - set(test.columns))
    print("%s dropped"% absent_columns)
    train.drop(absent_columns, axis=1, inplace=True)

drop_not_present(train, test)

['ГП - Общий прирост Qн', 'ТП(ИДН) Дебит жидкости', 'Qж, т/сут', 'ТП(ИДН) Дебит жидкости скорр-ый', 'Жидкость, т', 'Вода, т', 'ТП(ГРП) Дебит жидкости', 'Жидкость, м3', 'ТП(ГРП) Дебит жидкости скорр-ый', 'Нефть, м3', 'Дебит жидкости'] dropped


In [26]:
len(train.columns)== len(test.columns)

True

In [27]:
def show_uniq_test_train(train, test):
    #check all values that have zero ans nan only
    for c in sorted(train.columns):
        un = train[c].unique()
        if len(un)<100:
            tun = test[c].unique()
            print("%s ;train: %s; test:%s"%(c, un, tun))
show_uniq_test_train(train, test)

ГТМ ;train: [ 1.  0. nan]; test:[ 1.  0. nan]
Метод ;train: ['ЭЦН' 'ЭЦН/ -' nan 'ЭЦН/БЕЗ СПОСОБА' 'ШГН' 'ФОН' 'ЭЦН/ФОН' 'ФОН/ЭЦН'
 'ЭЦН/ШГН' 'ШГН/ЭЦН']; test:['ЭЦН' nan 'ФОН/ЭЦН' 'ЭЦН/ФОН' 'ШГН']
Характер работы ;train: ['НЕФ' 'НЕФ/НАГ' 'НАГ' nan]; test:['НЕФ' nan]
Состояние ;train: ['РАБ.' 'ОСТ.' 'ОСВ ТГ' 'Б/Д ТГ' 'Б/Д ПР Л' nan 'ОЖ ОСВ ТГ' 'НАК' 'ПЬЕЗ']; test:['РАБ.' nan 'НАК' 'Б/Д ТГ' 'ПЬЕЗ']
Время накопления ;train: [  0.  nan  60. 360. 372. 336. 217. 150. 155. 104. 196. 300. 310. 280.
 137. 221. 589. 551. 570. 165. 324. 356. 176. 240. 248. 232. 224. 496.
 464. 480. 320. 341. 319. 330. 371. 448.  16.  84. 440. 450. 465. 184.
 434. 420. 112. 120. 260. 478. 390.  26. 216. 392. 322. 348.  24. 373.
  91.   9. 279. 270.  73. 620. 600. 560. 128. 376. 349. 204.  70. 180.
 116. 527. 510. 143. 228. 364. 406. 210. 203. 252. 403.  52. 192.]; test:[  0.  nan  16. 149. 360. 300.]
Природный газ, м3 ;train: [ 0. nan]; test:[ 0. nan]
Газ из газовой шапки, м3 ;train: [ 0. nan]; test:[ 0. nan]
Конде

In [35]:
def split_continious_date_categorical_text(df):
    group_id = ["Скважина"]
    text = ["Причина простоя",
            "Куст",
            "Состояние на конец месяца",
            "Причина простоя.1",
            "Мероприятия",
            "Примечание"]
    categorical = ["Тип скважины",
                   "Неустановившийся режим",
                   "ГТМ",
                   "Метод",
                   "Характер работы",
                   "Состояние",
                   "Пласт МЭР", 
                   "Способ эксплуатации", 
                   "Тип насоса", 
                   "Состояние на конец месяца", 
                   "Номер бригады", 
                   "Фонтан через насос", 
                   "Нерентабельная",
                   "Назначение по проекту",
                   "Группа фонда",
                   "Тип дополнительного оборудования",
                   "Марка ПЭД",
                   "Тип ГЗУ",
                   "ДНС",
                   "КНС",
                   #useless potentially
                   "Диаметр плунжера",
                   "Природный газ, м3",
                   "Конденсат, т",
                   "Длина хода плунжера ШГН",
                   "Коэффициент подачи насоса",
                   "Дебит конденсата",
                   "Вязкость воды в пластовых условиях",
                   "Газ из газовой шапки, м3",
                   "Число качаний ШГН",
                   "Коэффициент сепарации",
                   "SKIN",
                   "КН закрепленный",
                   # radically different
                   "Радиус контура питания"
                   ]
    dates = ["Дата",
             "Дата_2",
             "Дата ГРП",
             "Время до псевдоуст-ся режима", 
             "Дата запуска после КРС", 
             "Дата пуска", 
             "Дата останова",
             "Дата ввода в эксплуатацию"]
    continious = list(set(df.columns) - set(dates) - set(categorical) - set(text) - set(group_id))
    return (df[group_id],df[continious], df[dates], df[categorical], df[text])

train_group, train_cont, train_dat, train_cat, train_text = split_continious_date_categorical_text(train)
test_group, test_cont, test_dat, test_cat, test_text = split_continious_date_categorical_text(test)

In [38]:
def get_object_columns(df):
    objects = []
    for c in df.columns:
        if df[c].dtype != pd.np.float:
            objects.append(c)
    return objects

def get_float(v):
    v = str(v)
    if v != "NaN":
        new = v.replace(",",".")
        return float(new)
    return v

def convert_locale_to_float(df):
    loc_float = get_object_columns(df)
    for c in loc_float:
        df.loc[:,c] = df[c].apply(get_float)
        
convert_locale_to_float(train_cont)
convert_locale_to_float(test_cont)

/anaconda2/envs/env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [40]:
def fill_with_mean(train, test):
    means=train.mean()
    norm_train = train.fillna(means)
    norm_test = test.fillna(means)
    return norm_train, norm_test
        
train_cont, test_cont = fill_with_mean(train_cont, test_cont)

In [42]:
train_cont.isnull().values.any() or test_cont.isnull().values.any()

False

In [43]:
#clear nans in target
indexes_to_delete = y_train[y_train.isnull()].index
y_train = y_train[~y_train.isnull()]
train_cont = train_cont.drop(index=indexes_to_delete)
y_train = y_train.apply(get_float)
print(len(y_train) == len(train_cont))

train_group = train_group.drop(index=indexes_to_delete)
print(len(y_train) == len(train_group))

train_cat = train_cat.drop(index=indexes_to_delete)
print(len(y_train) == len(train_cat))

True
True
True


In [45]:
len(y_train)

5489

In [46]:
len(train_cont)

5489

In [47]:
def transform_cats_to_labels(train_cat, test_cat):
    transformed_df = train_cat.copy()
    trans_test = test_cat.copy()
    for c in train_cat.columns:
        encoder = LabelEncoder()
        column_train = train_cat[c].astype(str)
        column_test = test_cat[c].astype(str)
        combined = pd.concat([column_train, column_test])
        encoder.fit(combined)
        transformed_df[c] = encoder.transform(column_train).reshape(-1,1)
        trans_test[c] = encoder.transform(column_test).reshape(-1,1)
    return transformed_df, trans_test

train_cat_labeled, test_cat_labeled = transform_cats_to_labels(train_cat, test_cat)

In [49]:
cb_regressor = CatBoostRegressor()
train_catboost_preds = cross_val_predict(cb_regressor, train_cat_labeled, y_train)
cb_regressor.fit(train_cat_labeled,y=y_train)
test_catboost_preds = pd.Series(cb_regressor.predict(test_cat_labeled), index=test_cat.index)

0:	learn: 592.9109833	total: 56.2ms	remaining: 56.1s
1:	learn: 586.6517816	total: 59.3ms	remaining: 29.6s
2:	learn: 580.7416001	total: 62.7ms	remaining: 20.8s
3:	learn: 575.2746745	total: 67.2ms	remaining: 16.7s
4:	learn: 569.8937369	total: 70.3ms	remaining: 14s
5:	learn: 564.6957020	total: 74.5ms	remaining: 12.3s
6:	learn: 559.7140962	total: 79.3ms	remaining: 11.2s
7:	learn: 555.5397389	total: 81.8ms	remaining: 10.1s
8:	learn: 550.8880049	total: 85.6ms	remaining: 9.42s
9:	learn: 546.8158277	total: 88.8ms	remaining: 8.79s
10:	learn: 542.8502321	total: 91.6ms	remaining: 8.24s
11:	learn: 539.0655423	total: 95.5ms	remaining: 7.86s
12:	learn: 535.7218944	total: 98.2ms	remaining: 7.45s
13:	learn: 532.6905378	total: 100ms	remaining: 7.05s
14:	learn: 529.3993053	total: 104ms	remaining: 6.82s
15:	learn: 526.4286562	total: 107ms	remaining: 6.56s
16:	learn: 523.4922446	total: 110ms	remaining: 6.38s
17:	learn: 520.6597165	total: 114ms	remaining: 6.2s
18:	learn: 517.9789346	total: 118ms	remaining:

191:	learn: 459.6231702	total: 844ms	remaining: 3.55s
192:	learn: 459.5965674	total: 852ms	remaining: 3.56s
193:	learn: 459.5621886	total: 860ms	remaining: 3.57s
194:	learn: 459.5134218	total: 869ms	remaining: 3.59s
195:	learn: 459.4975218	total: 874ms	remaining: 3.58s
196:	learn: 459.4328481	total: 880ms	remaining: 3.58s
197:	learn: 459.3908770	total: 884ms	remaining: 3.58s
198:	learn: 459.3863856	total: 886ms	remaining: 3.57s
199:	learn: 459.3590183	total: 891ms	remaining: 3.56s
200:	learn: 459.3452744	total: 894ms	remaining: 3.55s
201:	learn: 459.3014625	total: 898ms	remaining: 3.55s
202:	learn: 459.2915412	total: 901ms	remaining: 3.54s
203:	learn: 459.2444674	total: 905ms	remaining: 3.53s
204:	learn: 459.1997961	total: 909ms	remaining: 3.53s
205:	learn: 459.1560164	total: 913ms	remaining: 3.52s
206:	learn: 459.0520991	total: 918ms	remaining: 3.52s
207:	learn: 459.0364272	total: 921ms	remaining: 3.5s
208:	learn: 458.9462761	total: 924ms	remaining: 3.5s
209:	learn: 458.9305551	total:

370:	learn: 455.5610251	total: 1.63s	remaining: 2.76s
371:	learn: 455.5373798	total: 1.63s	remaining: 2.76s
372:	learn: 455.5066125	total: 1.64s	remaining: 2.76s
373:	learn: 455.4903067	total: 1.65s	remaining: 2.76s
374:	learn: 455.4737823	total: 1.65s	remaining: 2.76s
375:	learn: 455.4608905	total: 1.66s	remaining: 2.76s
376:	learn: 455.4339421	total: 1.67s	remaining: 2.75s
377:	learn: 455.4183223	total: 1.67s	remaining: 2.75s
378:	learn: 455.4027494	total: 1.67s	remaining: 2.74s
379:	learn: 455.3884203	total: 1.68s	remaining: 2.74s
380:	learn: 455.3766567	total: 1.68s	remaining: 2.74s
381:	learn: 455.3702460	total: 1.69s	remaining: 2.73s
382:	learn: 455.3185356	total: 1.69s	remaining: 2.73s
383:	learn: 455.3071249	total: 1.7s	remaining: 2.72s
384:	learn: 455.2772860	total: 1.7s	remaining: 2.72s
385:	learn: 455.2362580	total: 1.71s	remaining: 2.72s
386:	learn: 455.2200115	total: 1.71s	remaining: 2.71s
387:	learn: 455.2200071	total: 1.72s	remaining: 2.71s
388:	learn: 455.1954373	total:

542:	learn: 453.7542525	total: 2.41s	remaining: 2.03s
543:	learn: 453.7542515	total: 2.41s	remaining: 2.02s
544:	learn: 453.7437154	total: 2.41s	remaining: 2.02s
545:	learn: 453.7437135	total: 2.42s	remaining: 2.01s
546:	learn: 453.7375829	total: 2.43s	remaining: 2.01s
547:	learn: 453.7375819	total: 2.43s	remaining: 2.01s
548:	learn: 453.7238013	total: 2.44s	remaining: 2.01s
549:	learn: 453.7180421	total: 2.46s	remaining: 2.01s
550:	learn: 453.7103042	total: 2.46s	remaining: 2.01s
551:	learn: 453.7103032	total: 2.47s	remaining: 2s
552:	learn: 453.6860992	total: 2.48s	remaining: 2s
553:	learn: 453.6819162	total: 2.48s	remaining: 2s
554:	learn: 453.6627551	total: 2.49s	remaining: 1.99s
555:	learn: 453.6563692	total: 2.49s	remaining: 1.99s
556:	learn: 453.6513391	total: 2.5s	remaining: 1.99s
557:	learn: 453.6513374	total: 2.5s	remaining: 1.98s
558:	learn: 453.6366122	total: 2.5s	remaining: 1.97s
559:	learn: 453.6257116	total: 2.51s	remaining: 1.97s
560:	learn: 453.6125229	total: 2.51s	rem

722:	learn: 452.4179975	total: 3.38s	remaining: 1.29s
723:	learn: 452.4082938	total: 3.38s	remaining: 1.29s
724:	learn: 452.3988393	total: 3.39s	remaining: 1.29s
725:	learn: 452.3932204	total: 3.4s	remaining: 1.28s
726:	learn: 452.3930148	total: 3.4s	remaining: 1.28s
727:	learn: 452.3891503	total: 3.41s	remaining: 1.27s
728:	learn: 452.3783616	total: 3.41s	remaining: 1.27s
729:	learn: 452.3656938	total: 3.42s	remaining: 1.26s
730:	learn: 452.3549695	total: 3.42s	remaining: 1.26s
731:	learn: 452.3506879	total: 3.43s	remaining: 1.25s
732:	learn: 452.3428176	total: 3.44s	remaining: 1.25s
733:	learn: 452.3367727	total: 3.44s	remaining: 1.25s
734:	learn: 452.3363723	total: 3.44s	remaining: 1.24s
735:	learn: 452.3340826	total: 3.45s	remaining: 1.24s
736:	learn: 452.3321335	total: 3.46s	remaining: 1.23s
737:	learn: 452.3320763	total: 3.46s	remaining: 1.23s
738:	learn: 452.3289092	total: 3.47s	remaining: 1.22s
739:	learn: 452.3287787	total: 3.47s	remaining: 1.22s
740:	learn: 452.3221925	total:

...

260:	learn: 410.8745235	total: 1.59s	remaining: 4.51s
261:	learn: 410.8481945	total: 1.6s	remaining: 4.52s
262:	learn: 410.8108420	total: 1.64s	remaining: 4.61s
263:	learn: 410.7836519	total: 1.68s	remaining: 4.68s
264:	learn: 410.7541173	total: 1.69s	remaining: 4.68s
265:	learn: 410.7293344	total: 1.69s	remaining: 4.67s
266:	learn: 410.7157678	total: 1.7s	remaining: 4.68s
267:	learn: 410.6900264	total: 1.71s	remaining: 4.68s
268:	learn: 410.6746764	total: 1.73s	remaining: 4.69s
269:	learn: 410.6404916	total: 1.74s	remaining: 4.7s
270:	learn: 410.6241385	total: 1.74s	remaining: 4.69s
271:	learn: 410.6207842	total: 1.75s	remaining: 4.68s
272:	learn: 410.5969242	total: 1.75s	remaining: 4.67s
273:	learn: 410.5651965	total: 1.76s	remaining: 4.67s
274:	learn: 410.5203496	total: 1.77s	remaining: 4.67s
275:	learn: 410.4856480	total: 1.77s	remaining: 4.65s
276:	learn: 410.4826718	total: 1.78s	remaining: 4.64s
277:	learn: 410.4448472	total: 1.79s	remaining: 4.64s
278:	learn: 410.4010175	total: 

436:	learn: 407.2073471	total: 2.99s	remaining: 3.86s
437:	learn: 407.1986738	total: 3s	remaining: 3.85s
438:	learn: 407.1875065	total: 3.01s	remaining: 3.85s
439:	learn: 407.1646840	total: 3.02s	remaining: 3.85s
440:	learn: 407.1530121	total: 3.04s	remaining: 3.85s
441:	learn: 407.1435194	total: 3.04s	remaining: 3.84s
442:	learn: 407.1342095	total: 3.05s	remaining: 3.83s
443:	learn: 407.1145027	total: 3.06s	remaining: 3.83s
444:	learn: 407.1077491	total: 3.06s	remaining: 3.82s
445:	learn: 407.0991607	total: 3.07s	remaining: 3.81s
446:	learn: 407.0958449	total: 3.07s	remaining: 3.8s
447:	learn: 407.0881840	total: 3.08s	remaining: 3.79s
448:	learn: 407.0704677	total: 3.09s	remaining: 3.79s
449:	learn: 407.0650386	total: 3.09s	remaining: 3.78s
450:	learn: 407.0422665	total: 3.1s	remaining: 3.77s
451:	learn: 407.0375204	total: 3.1s	remaining: 3.76s
452:	learn: 407.0276336	total: 3.11s	remaining: 3.75s
453:	learn: 407.0168320	total: 3.11s	remaining: 3.74s
454:	learn: 407.0099131	total: 3.1

591:	learn: 406.0738340	total: 4.19s	remaining: 2.89s
592:	learn: 406.0666213	total: 4.2s	remaining: 2.88s
593:	learn: 406.0605784	total: 4.21s	remaining: 2.88s
594:	learn: 406.0554279	total: 4.22s	remaining: 2.87s
595:	learn: 406.0490790	total: 4.22s	remaining: 2.86s
596:	learn: 406.0464417	total: 4.23s	remaining: 2.86s
597:	learn: 406.0442876	total: 4.24s	remaining: 2.85s
598:	learn: 406.0433511	total: 4.24s	remaining: 2.84s
599:	learn: 406.0428862	total: 4.25s	remaining: 2.83s
600:	learn: 406.0379556	total: 4.25s	remaining: 2.82s
601:	learn: 406.0364877	total: 4.26s	remaining: 2.81s
602:	learn: 406.0349264	total: 4.26s	remaining: 2.81s
603:	learn: 406.0306574	total: 4.27s	remaining: 2.8s
604:	learn: 406.0300765	total: 4.27s	remaining: 2.79s
605:	learn: 406.0221938	total: 4.28s	remaining: 2.78s
606:	learn: 406.0197013	total: 4.28s	remaining: 2.77s
607:	learn: 406.0120747	total: 4.29s	remaining: 2.77s
608:	learn: 406.0119865	total: 4.29s	remaining: 2.76s
609:	learn: 406.0089165	total:

752:	learn: 405.5742240	total: 5.58s	remaining: 1.83s
753:	learn: 405.5722802	total: 5.6s	remaining: 1.83s
754:	learn: 405.5687667	total: 5.61s	remaining: 1.82s
755:	learn: 405.5655895	total: 5.63s	remaining: 1.82s
756:	learn: 405.5619129	total: 5.64s	remaining: 1.81s
757:	learn: 405.5607874	total: 5.66s	remaining: 1.81s
758:	learn: 405.5595812	total: 5.67s	remaining: 1.8s
759:	learn: 405.5585665	total: 5.68s	remaining: 1.79s
760:	learn: 405.5552299	total: 5.7s	remaining: 1.79s
761:	learn: 405.5531371	total: 5.71s	remaining: 1.78s
762:	learn: 405.5515126	total: 5.72s	remaining: 1.78s
763:	learn: 405.5478893	total: 5.74s	remaining: 1.77s
764:	learn: 405.5465760	total: 5.75s	remaining: 1.77s
765:	learn: 405.5434430	total: 5.76s	remaining: 1.76s
766:	learn: 405.5434393	total: 5.77s	remaining: 1.75s
767:	learn: 405.5427763	total: 5.79s	remaining: 1.75s
768:	learn: 405.5393737	total: 5.8s	remaining: 1.74s
769:	learn: 405.5387010	total: 5.81s	remaining: 1.74s
770:	learn: 405.5381822	total: 5

923:	learn: 405.2750332	total: 6.97s	remaining: 574ms
924:	learn: 405.2748165	total: 6.98s	remaining: 566ms
925:	learn: 405.2737441	total: 6.99s	remaining: 558ms
926:	learn: 405.2733923	total: 6.99s	remaining: 551ms
927:	learn: 405.2710998	total: 7s	remaining: 543ms
928:	learn: 405.2701116	total: 7s	remaining: 535ms
929:	learn: 405.2669370	total: 7.01s	remaining: 528ms
930:	learn: 405.2667470	total: 7.01s	remaining: 520ms
931:	learn: 405.2659158	total: 7.02s	remaining: 512ms
932:	learn: 405.2618129	total: 7.02s	remaining: 504ms
933:	learn: 405.2608788	total: 7.03s	remaining: 497ms
934:	learn: 405.2605778	total: 7.04s	remaining: 489ms
935:	learn: 405.2586522	total: 7.04s	remaining: 481ms
936:	learn: 405.2586511	total: 7.05s	remaining: 474ms
937:	learn: 405.2560957	total: 7.06s	remaining: 467ms
938:	learn: 405.2560947	total: 7.07s	remaining: 459ms
939:	learn: 405.2560936	total: 7.07s	remaining: 451ms
940:	learn: 405.2560926	total: 7.08s	remaining: 444ms
941:	learn: 405.2532615	total: 7.0

In [50]:
def get_train_mixture():
    return train_catboost_preds
def get_test_mixture():
    return test_catboost_preds

In [51]:
# now we have clear non-normalized data, let's normalize first
def normalize(train, test):
    scaler = StandardScaler()
    norm_train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index = train.index)
    norm_test = pd.DataFrame(scaler.transform(test), columns=test.columns, index = test.index)
    return norm_train, norm_test

train_cont, test_cont = normalize(train_cont, test_cont)

In [52]:
def get_ensemble():
    ridge = Ridge()
    rtree = RandomForestRegressor(n_jobs=-1, n_estimators=50)
    svr = LinearSVR()
    return [ridge, rtree, svr]

def get_columns():
    return ["ridge", "rtree", "svr", "catboost"]

In [53]:
def get_meta_train_preds(X, y):
    predicts = []
    for cl in get_ensemble():
        predicts.append(cross_val_predict(cl, X,y, n_jobs=-1))
    predicts.append(get_train_mixture())
    return pd.DataFrame(np.vstack(predicts).transpose(), index=y.index, columns=get_columns())

def get_meta_test_predict(X_train, y_train, X_test):
    test_predicts = []
    for cl in get_ensemble():
        print(cross_val_score(cl, X_train, y_train, n_jobs=-1, scoring="neg_mean_absolute_error"))
        cl.fit(X_train, y_train)
        pr = cl.predict(X_test)
        test_predicts.append(pr)
    test_predicts.append(get_test_mixture())
    return  pd.DataFrame(np.vstack(test_predicts).transpose(), index=X_test.index, columns=get_columns())

def get_stacked_ensemble_predict(X_meta, y, X_test):
    regressor = xgb.XGBRegressor()
    regressor.fit(X_meta, y)
    return pd.DataFrame(regressor.predict(X_test), index=X_test.index)

def get_first_tower_prediction(train, y, test):
    X_meta_train = get_meta_train_preds(train, y)
    X_meta_test = get_meta_test_predict(train, y, test)
    return get_stacked_ensemble_predict(X_meta_train, y, X_meta_test)

In [55]:
def get_n_item_index(group_size, df, group):
    new_df = pd.concat([df, group], axis = 1)
    index = []
    group = new_df.groupby(["Скважина"])
    for name, group in group:
        if len(group)<group_size:
            continue
        for start in range(len(group.index) - group_size):
            gr =group.index[start:start+group_size]
            index.append(gr)
    return index

def get_timed_ds(meta_size, df, group, y):
    if meta_size >= 1:
        meta_indexes = get_n_item_index(meta_size, df, group)
        first_value_idx = []
        timed_ds = df.copy()
        metas = []
        columns = []
        for i in range(meta_size):
            columns.append("meta%s"%str(i))
        for a in meta_indexes:
            first_value_idx.append(a[0])
            metas.append(list(y.loc[a]))
        metas_df = pd.DataFrame.from_records(metas, index=first_value_idx, columns=columns)
        return pd.concat([timed_ds.loc[first_value_idx], metas_df], axis=1)
    elif meta_size == 0:
        return df
    
def get_n_tower_predictions(n, train, y, test, train_group):
    X_meta_train = get_meta_train_preds(train, y)
    X_meta_test = get_meta_test_predict(train, y, test)
    test_predictions= []
    for i in range(0,n):
        train_timed_ds = get_timed_ds(i,X_meta_train, train_group, y)
        y_timed = y.loc[train_timed_ds.index]
        test_predict = get_stacked_ensemble_predict(train_timed_ds, y_timed,X_meta_test)
        test_predictions.append(test_predict)
        X_meta_test["meta%s"%i]=test_predict
    return pd.concat(test_predictions,axis=1)

In [56]:
time_serie_pred = get_n_tower_predictions(6, train_cont, y_train,test_cont, train_group)

[ -81.24202476  -90.79687321 -132.14470966]
[-40.03138787 -55.96297399 -58.63667151]
[-84.38120236 -82.42176398 -96.79284019]


In [63]:
final_time_serie_pred = time_serie_pred.values.reshape(-1,1)

In [59]:
len(final_time_serie_pred)

1014

In [64]:
456.2430 - np.mean(final_time_serie_pred)

41.649127929687495

In [65]:
final_time_serie_pred = final_time_serie_pred+ (456.2430 - np.mean(final_time_serie_pred))


In [66]:
final_time_serie = pd.Series(np.squeeze(final_time_serie_pred))

In [67]:
final_time_serie.to_csv("task2_base.csv",header=["_VAL_"],index_label=["_ID_"])